# Lab HW 5

### **Question: In the context of neural machine translation (NMT), consider a sequence-to-sequence LSTM model translating from English to Spanish. You are asked to evaluate the model performance under different settings:**

> **Train the model both with and without attention mechanisms.**

> **During inference, experiment with greedy decoding vs. beam search decoding.**

> **Compare the results with and without teacher forcing during training.**

> **For each of the following combinations, describe how the model's performance might change and explain why:**

>>> *No attention, greedy decoding, with teacher forcing*

>>> *Attention, beam search decoding, with teacher forcing*

>>> *Attention, greedy decoding, without teacher forcing*

>>> *No attention, beam search decoding, without teacher forcing*

In [ ]:
import tensorflow as tf
import os
import pathlib
import random
import string
import re
import numpy as np
import tensorflow.data as tf_data
import tensorflow.strings as tf_strings
import keras
from keras import layers
from keras.layers import TextVectorization

In [ ]:
Bisen = []

# Download the dataset
data_file = keras.utils.get_file( fname="spa-eng.zip", origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip", extract=True )
data_file = pathlib.Path( data_file ).parent / "spa-eng" / "spa.txt"
# Prepare the dataset
with open(data_file) as f:
    lines = f.read().split("\n")[:-1]

for line in lines:
    sen_en, sen_sp = line.split("\t");    sen_sp = "[start] " + sen_sp + " [end]"
    Bisen.append( (sen_en, sen_sp) )

random.shuffle(Bisen)
num_val_samples = int(0.15 * len(Bisen))
num_train_samples = len(Bisen) - 2 * num_val_samples
train_pairs = Bisen[:num_train_samples]
val_pairs = Bisen[num_train_samples:num_train_samples + num_val_samples]
test_pairs = Bisen[(num_train_samples + num_val_samples):]

# Text Vectorization
We'll vectorize the English and Spanish sentences, limiting the vocabulary size and sequence length.


In [ ]:
# Vectorizing the text data
strip_chars = string.punctuation + "¿"; strip_chars = strip_chars.replace("[", "").replace("]", "")
VSIZE = 15000;  sequence_length = 20
BSIZE = 64

def standardization(input_string):
    lowercase = tf_strings.lower(input_string)
    return tf_strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")

# English and spanish vectorisation
vect_en = TextVectorization(  max_tokens=VSIZE,  output_mode="int",  output_sequence_length=sequence_length )
vect_sp = TextVectorization(  max_tokens=VSIZE,  output_mode="int",   output_sequence_length=sequence_length + 1, standardize=standardization )

# Adapt vectorization to the training data
train_eng_texts = [pair[0] for pair in train_pairs];  train_spa_texts = [pair[1] for pair in train_pairs]
vect_en.adapt(train_eng_texts); vect_sp.adapt(train_spa_texts)


# Formatting the dataset for the model
We format the dataset into pairs of encoder inputs (English) and decoder inputs (Spanish), which will be used for training.


In [ ]:
def format_dataset(sen_en, sen_sp):
    vec_en = vect_en(sen_en); vec_sp = vect_sp(sen_sp)
    return ({ "encoder_inputs": vec_en, "decoder_inputs": vec_sp[:, :-1], }, vec_sp[:, 1:] )

def make_dataset(pairs):
    line_en, line_sp = zip(*pairs)
    line_en = list(line_en); line_sp = list(line_sp)

    # slicing the dataset
    dataset = tf_data.Dataset.from_tensor_slices((line_en, line_sp)); dataset = dataset.batch(BSIZE)

    # applying the format function on each of the dataset's row
    return dataset.map(format_dataset).cache().shuffle(2048).prefetch(16)

ds_train = make_dataset(train_pairs); ds_vali = make_dataset(val_pairs)

# Transformer Model

In [ ]:
class PEmbedding(layers.Layer):

    def __init__(self, sequence_length, vsize, embed_dim, **kwargs):

        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=vsize, output_dim=embed_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=embed_dim)
        self.sequence_length = sequence_length
        self.vocab_size = vsize
        self.embed_dim = embed_dim

    def call(self, inputs):

        ln = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=ln, delta=1)
        embedded_tokens = self.token_embeddings(inputs);  embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
      return None if mask is None else tf.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({ "sequence_length": self.sequence_length,  "vocab_size": self.vocab_size,  "embed_dim": self.embed_dim })
        return config

class TransEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        attention_output = self.attention(query=inputs, value=inputs, key=inputs)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

class TransDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, use_attention=True, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.use_attention = use_attention
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([
            layers.Dense(latent_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        padding_mask = tf.minimum(tf.cast(mask[:, None, :], dtype="int32"), causal_mask) if mask is not None else None

        attention_output_1 = self.attention_1(query=inputs, value=inputs, key=inputs, attention_mask=causal_mask)
        out_1 = self.layernorm_1(inputs + attention_output_1)

        if self.use_attention:
            attention_output_2 = self.attention_2(
                query=out_1, value=encoder_outputs, key=encoder_outputs, attention_mask=padding_mask
            )
            out_2 = self.layernorm_2(out_1 + attention_output_2)
        else:
            out_2 = out_1

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, None]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        return tf.tile(tf.reshape(mask, (1, sequence_length, sequence_length)), [batch_size, 1, 1])

def build_trans_model(use_attention=True):
    embed_dim = 256
    latent_dim = 2048
    num_heads = 8

    # Encoder
    encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
    x = PEmbedding(sequence_length, VSIZE, embed_dim)(encoder_inputs)
    encoder_outputs = TransEncoder(embed_dim, latent_dim, num_heads)(x)
    encoder = keras.Model(encoder_inputs, encoder_outputs)

    # Decoder
    decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
    encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
    x = PEmbedding(sequence_length, VSIZE, embed_dim)(decoder_inputs)

    if use_attention:
        x = TransDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
    else:
        x = layers.Dense(latent_dim, activation="relu")(x)

    x = layers.Dropout(0.5)(x)
    decoder_outputs = layers.Dense( VSIZE, activation="softmax")(x)
    decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

    # Assembling full transformer model
    decoder_outputs = decoder([decoder_inputs, encoder_outputs])
    transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs, name="transformer")
    return transformer, encoder, decoder


# Evaluating the Model on 4 different settings



In [ ]:
epochs = 2

max_ln = 20
spa_vocab = vect_sp.get_vocabulary(); spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))

In [ ]:
def decode_sequence(transformer, input_sentence, use_greedy=True, beam_size=3):
    tokenized_input_sentence = vect_en([input_sentence]);  decoded_sentence = "[start]"
    for i in range(max_ln):
        tokenized_target_sentence = vect_sp([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        if use_greedy:
            sampled_token_index = tf.argmax(predictions[0, i, :]).numpy()
        else:
            top_k_indices = tf.argsort(predictions[0, i, :])[-beam_size:].numpy()
            sampled_token_index = random.choice(top_k_indices)

        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence



def calculate_model(use_attention, use_greedy, beam_size=3, teacher_forcing=True):


    transformer, encoder, decoder = build_trans_model(use_attention=use_attention)
    transformer.compile( optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"] )
    transformer.fit(ds_train, epochs=epochs, validation_data=ds_vali)

    if teacher_forcing:
        history = transformer.fit(ds_train, epochs=epochs, validation_data=ds_vali)
    else:
        history = transformer.fit(ds_train.map(lambda x, y: (x, x["decoder_inputs"])), epochs=epochs, validation_data=ds_vali)
    test_eng_texts = [pair[0] for pair in test_pairs]
    translations = []
    for _ in range(6):
        input_sentence = random.choice(test_eng_texts)
        translated_sentence = decode_sequence(transformer, input_sentence, use_greedy=use_greedy, beam_size=beam_size)
        translations.append((input_sentence, translated_sentence))

    return history.history, translations

print("No attention, greedy decoding with the use of teacher forcing")
history_1, translations_1 = calculate_model(use_attention=False, use_greedy=True, teacher_forcing=True)

print("2. Attention, beam search decoding with the use of teacher forcing")
history_2, translations_2 = calculate_model(use_attention=True, use_greedy=False, beam_size=3, teacher_forcing=True)

print("3. Attention, greedy decoding, without the use of teacher forcing")
history_3, translations_3 = calculate_model(use_attention=True, use_greedy=True, teacher_forcing=False)

print(f"{history_1}\n{history_2}\n{history_3}")